In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/CMU_DataScience/master/lung.csv?fbclid=IwAR2qBGEAlrnrn6Novzqsv6GbvFiSkREQ4DO4o5icUV6JLCFVfXkmPlQNMUA', sep=',', header = None)

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.isnull().any()

In [ ]:
df_clean = df

In [ ]:
Train_class = np.unique(df_clean[0])
Train_class

In [ ]:
Train_class = [sum(df[0]==Train_class[0]),
               sum(df[0]==Train_class[1])]
Train_class

In [ ]:
y = df_clean[0].values
X = df_clean.drop([0],axis = 1).values


In [ ]:
np.shape(y)

In [ ]:
np.shape(X)

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=1)
X, y = sm.fit_resample(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=0)

In [ ]:
np.shape(X_train)

In [ ]:
np.shape(X_test)

In [ ]:
#from sklearn.decomposition import PCA
#fr = PCA(n_components=150)
#fr.fit(X_train)
#X_train = fr.transform(X_train)
#X_test = fr.transform(X_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

In [ ]:
np.max(X_train_norm,0)

In [ ]:
np.max(X_test_norm,0)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
params = {
    'C' : [2,4,6,8,10,12], 
    'gamma' : [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32] 
}
clf = GridSearchCV(SVC(),params, cv=10)
clf.fit(X_train_norm, y_train)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

In [ ]:
y_predict = clf.predict(X_test_norm)
print("Test accuracy : "+str(sum(y_test == y_predict)/len(y_test)*100))

In [ ]:
df_t= pd.read_csv('https://raw.githubusercontent.com/plenoi/CMU_DataScience/master/lung_test.csv?fbclid=IwAR1qt2Sg8_yKAaZ6n1j-l6T9kt6FFMDw6NSzy5FHNMJFqv3uKlz4EItFg5Q', sep=',', header = None)

In [ ]:
yt = df_t[0].values
Xt = df_t.drop([0],axis = 1).values

In [ ]:
X_test_norm = scaler.transform(Xt)

In [ ]:
y_predict = clf.predict(X_test_norm)
print("Test accuracy : "+str(sum(yt == y_predict)/len(yt)*100))

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
clf4 = SVC(probability=True)

eclf = VotingClassifier(

estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),('scv',clf4)],voting='soft')

params = {'lr__C': [1.0, 100.0], 'rf__n_estimators': [20, 200],'scv__C': [2,4,6,8,10,12]}

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=10)

grid = grid.fit(X_train_norm,y_train)

In [ ]:
y_predict = grid.predict(X_test_norm)
print("Test accuracy : "+str(sum(y_test == y_predict)/len(y_test)*100))